![Podmanlogo](Pictures/podman-logo.png)

# Managing multiple containers

Running a single container with podman is just fine, we have demonstrated how simple it is througout this workshop. But, what happens if I want to run multiple containers simultaneously? Do you need to execute them one by one?

You can use the tool podman compose to execute multiple containers in a single command. Podman compose is a tool that helps you defining the list of containers and all of their configuration in a text file written in yaml format. Then you can choose to execute, stop, remove, update all of the containers defined in the file at once. Podman compose is a very powerful tool but it's limited to execute on a single system.

Lets run a basic example of podman-compose. First we are going to build two container images, one of them will be an nginx proxy that will be exposed through a port of our host and the second one will be simulating a backend. The backend is just an httpd server showing the message "I'm the backend", but what we want to demonstrate here is how to deploy multi-tier applications and that is why the nginx-proxy will be acting as a front-end redirecting all the traffic in a specific location to the backend.


We begin by building our backend container image. As we have seen in the section two of this workshop we will need a Containerfile and an index.html file. First we will create the Containerfile:

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Podman101'] }}

In [ ]:
HostIP={{ hostvars[inventory_hostname]['IP-WKSHP-Podman101'] }}

In [ ]:
cat > Containerfile-backend << EOF
FROM docker.io/redhat/ubi9
RUN dnf -y update \
 && dnf -y install httpd \
 && dnf clean all
COPY index.html /var/www/html/index.html
EXPOSE 80
ENTRYPOINT /usr/sbin/httpd -DFOREGROUND
EOF
cat Containerfile-backend

Now the index.html file:

In [ ]:
echo "I’m the backend" > index.html
cat index.html

Once we have that we can go ahead and build the backend container.

In [ ]:
podman build --tag myhttpd:v1 -f ./Containerfile-backend

Lets build the frontend, an nginx container that will redirect all the traffic pointing to the /backend location to our backend. This frontend is using nginx technology. Nginx will also be able to show a message if we don't point to any location, the message will be "I'm just a proxy!", for that we need a dedicated html file.

In [ ]:
echo "I'm just a proxy!" > nginx-index.html
cat nginx-index.html

Also we want to redirect the traffic going to the location /backend to the backend container. We need to provide a default.conf file so nginx knows about this traffic redirection. If you're not familiar with nginx don't worry too much about this file.

In [ ]:
cat << EOF > default.conf
server {
	listen   	80;
	listen  [::]:80;
	server_name  localhost;

	location / {
    	root   /usr/share/nginx/html;
    	index  index.html index.htm;
	}
    
	location /backend/ {
    	proxy_pass http://backend_container/;
	}

	error_page   500 502 503 504  /50x.html;
	location = /50x.html {
    	root   /usr/share/nginx/html;
	}
}
EOF
cat default.conf

Last thing we need to do is creating the Containerfile for our frontend. This Containerfile uses the nginx base image and will copy the two files we just created to the nginx container image in the appropiate directories.

In [ ]:
cat << EOF > Containerfile-nginx
FROM nginx:latest
COPY default.conf /etc/nginx/conf.d/default.conf
COPY nginx-index.html /usr/share/nginx/html/index.html
EOF
cat Containerfile-nginx

Now we can build the container image for the front end.

In [ ]:
podman build -t nginx-reverse-proxy -f ./Containerfile-nginx

Check both your images have been created, they should appear as "localhost/nginx-reverse-proxy" and "localhost/myhttpd":

In [ ]:
podman images

Now, we can create a compose file and create both containers at once. First lets create the file and then we will see what's in it.

In [ ]:
cat > compose.yml << EOF
services:
  nginx-proxy:
        container_name: nginx_proxy_workshop
        image: "localhost/nginx-reverse-proxy:latest"
        ports:
          - "{{ HTTPPORT }}:80"
        networks:
          - "workshop"
        depends_on:
          - backend
  backend:
        container_name: backend_container
        image: "localhost/myhttpd:v1"
        networks:
          - "workshop"
networks:
  workshop:
EOF
cat compose.yml

As you can see the file that we just created is using the Yaml format, meaning that indentation defines what objects are at the same level or what lines are options inside certain object.

In our example, as you can see in the output of previous command, the file begins with a line that states "services" and then we have two lines "nginx-proxy" and "backend", as the last two lines are indented inside the "services" section podman-compose will know that they are service objects. Then we have a few options within each service that define how they will be deployed. Both "nginx-proxy" and "backend" are arbitrary names.
You can also see that we created a dedicated network called workshop for our services.

Within each service we define the container image to be used with the "image" option, the container name with "container_name" option, the ports to be exposed (if any) with the "ports" option and the network to be used with the "networks" option. How podman networks work is not covered in this workshop, it will be explained in a Podman 201 workshop in the future. 

In the nginx-proxy service we defined the option "depends_on" stating that nginx-proxy will only be started once the service backend has started. This is the way of managing micro services boot dependencies with podman-compose.

Now we can just run the command to execute both containers.

In [ ]:
podman-compose up -d

Now check both of your containers are running.

In [ ]:
podman ps

You see how easy it is to create multiple containers with podman-compose.

We can test if everything works, first lets test if we can reach the nginx proxy:

In [ ]:
curl -s localhost:{{ HTTPPORT }}

We reached the frontend proxy, now lets reach our backend:

In [ ]:
curl -s localhost:{{ HTTPPORT }}/backend/

As you can see we have accessed the backend container without needing to expose it through podman, thanks to using another container as proxy we can redirect traffic from it just by using the location "/backend/". This way of working is very common and forward proxies are widely used for multi-tiered applications. This increases simplicity when accesing containers and also security as you're not exposing your application directly and you can use a secured protocol (like https) for your proxy.

Now we can stop both containers at a single time:

In [ ]:
podman-compose down

Check they have been stopped.

In [ ]:
podman ps

Podman-compose is a very interesting tool, but it's limited to deploying containers to a single system. If you're looking at deploying your containers in multiple systems simultaneously, looking for scalability and high availability, then you will most probably end up using kubernetes as it has become the standard platform for running containerized workloads in the enterprise space.
Kubernetes, same as podman-compose, uses yaml files to define the desired state of your containerized workloads and, once you pass the file to the control plane nodes of kubernetes, it will make sure your containers run according to your specifications. Furthermore they will be spread accross all the nodes that are part of your kubernetes cluster.

![k8slogo](Pictures/kubernetes-logo.png)

Now, transitioning from single system container engines like Podman or Docker to kubernetes may not always be easy. That is why Podman can generate a kubernetes yaml file from the container that you are running righ now with Podman. First you just need to execute a new container:

In [ ]:
podman run -d --rm --name=kubepodman docker.io/redhat/ubi9 sleep 999

Now your container is running:

In [ ]:
podman ps

With this, you can execute the "podman generate kube" command to export a yaml file, which you can use as definition for your kubernetes environment.

In [ ]:
podman generate kube kubepodman > kubepodman.yml
cat kubepodman.yml

The output of the previous command is the content of the yaml file that you need to use for running your container in a kubernetes workload. At the beginning of the file you can even see a few comments in which you find the command you need to run in order to start executing this container in your kubernetes cluster: "kubectl create -f kubepodman.yml".

As you can see, podman has multiple tools that allow you to transition to managing multiple containers at once and simplify the transition to those tools.

# Cleanup

In [ ]:
podman stop --all
podman rm --all
podman rmi docker.io/redhat/ubi9 docker.io/library/nginx localhost/nginx-reverse-proxy localhost/myhttpd:v1
rm kubepodman.yml compose.yml Containerfile-backend Containerfile-nginx default.conf index.html nginx-index.html

In [ ]:
%logout

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps


<h2>Conclusion&nbsp;&nbsp;&nbsp;&nbsp;<a href="6-WKSHP-Conclusion.ipynb" target="New" title="Next LAB: Conclusion"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="5-WKSHP-Managing-multiple-containers.ipynb" target="New" title="Back: Managing multiple containers"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="6-WKSHP-Conclusion.ipynb" target="New" title="Next: Conclusion"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>